# Chapter-9 Standard Libraries
This notebook contains the sample source code explained in the book *Hands-On Julia Programming, Sambit Kumar Dash, 2021, bpb Publications. All Rights Reserved*.

In [1]:
using Pkg
pkg"activate ."
pkg"instantiate"

  Activating environment at `C:\Bahiyaa\Julia-Assignment\Chapter 09\Project.toml`


## 9.1 Introduction

There are various functions and methods that are part of Julia already. A developer needs to use them directly rather than having to code for them. This chapter is a bird's eyeview of such methods. 

### Modules

Namespaces where a method or a type reside. Using a method or type outside the module may require explicit qualification. 

In [2]:
module n1
    function f()
        println("Defined as n1.f")
    end
    function k()
        println("Defined as n1.k")
    end
    function t()
        println("Defined as n1.t")
    end
    export f
end

module n2
    function f()
        println("Defined as n2.f")
    end
    function k()
        println("Defined as n2.k")
    end
    export k
end

Main.n2

In [3]:
names(n1)

2-element Vector{Symbol}:
 :f
 :n1

In [4]:
names(n2)

2-element Vector{Symbol}:
 :k
 :n2

In [5]:
n1.f()

Defined as n1.f


In [6]:
n2.f()

Defined as n2.f


In [7]:
using .n1

In [8]:
f()

Defined as n1.f


In [9]:
using Main.n2

In [10]:
k()

Defined as n2.k


In [11]:
using .n1: k

In [12]:
k()

Defined as n2.k


In [13]:
using .n1: t

In [14]:
t()

Defined as n1.t


## 9.2 Standard Modules

`Core`, `Base` and `Main` the bare minimum modules of a Julia shell. `m1` and `m2` here are submodules of `Main`.

In [15]:
varinfo()

| name |      size | summary |
|:---- | ---------:|:------- |
| Base |           | Module  |
| Core |           | Module  |
| Main |           | Module  |
| n1   | 6.450 KiB | Module  |
| n2   | 5.577 KiB | Module  |


In [16]:
y = 7

7

In [17]:
varinfo()

| name |      size | summary |
|:---- | ---------:|:------- |
| Base |           | Module  |
| Core |           | Module  |
| Main |           | Module  |
| n1   | 6.450 KiB | Module  |
| n2   | 5.577 KiB | Module  |
| y    |   8 bytes | Int64   |


## 9.3 System

The submodule that interacts with the operating system. System command execution being one of them. 

In [18]:
run(`cmd /c dir`)

 Volume in drive C is OS
 Volume Serial Number is 3805-1403

 Directory of C:\Bahiyaa\Julia-Assignment\Chapter 09

29-09-2022  19:27    <DIR>          .
10-09-2022  15:11    <DIR>          ..
29-09-2022  19:07    <DIR>          .ipynb_checkpoints
29-09-2022  19:27            67,866 Chapter-9 Standard Libraries.ipynb
29-09-2022  19:07             5,151 Manifest.toml
29-09-2022  19:07                55 Project.toml
               3 File(s)         73,072 bytes
               3 Dir(s)  317,737,959,424 bytes free


Process(`cmd /c dir`, ProcessExited(0))

In [19]:
Sys.isunix(), Sys.iswindows()

(false, true)

## 9.4 Filesystem

File organization and manipulation (not read and write). 

In [20]:
old_dir = pwd()

"C:\\Bahiyaa\\Julia-Assignment\\Chapter 09"

In [21]:
cd("..")

In [22]:
pwd()

"C:\\Bahiyaa\\Julia-Assignment"

In [23]:
cd(old_dir)

In [24]:
pwd()

"C:\\Bahiyaa\\Julia-Assignment\\Chapter 09"

In [25]:
cd(".."); pwd()

"C:\\Bahiyaa\\Julia-Assignment"

In [26]:
readdir()  #Same as readdir(pwd())

21-element Vector{String}:
 ".DS_Store"
 ".git"
 "Chapter 02"
 "Chapter 03"
 "Chapter 04"
 "Chapter 05"
 "Chapter 06"
 "Chapter 07"
 "Chapter 08"
 "Chapter 09"
 "Chapter 10"
 "Chapter 11"
 "Chapter 12"
 "Chapter 13"
 "Chapter 14"
 "Chapter 15"
 "Chapter 16"
 "Chapter 17"
 "LICENSE"
 "README.md"
 "src"

In [27]:
for (root, dirs, files) in walkdir("Chapter-09")
    println("Directories in $root")
    for dir in dirs
        println(joinpath(root, dir)) # path to directories
    end
    println("Files in $root")
    for file in files
        println(joinpath(root, file)) # path to files
    end
end

LoadError: IOError: readdir("Chapter-09"): no such file or directory (ENOENT)

In [28]:
cd(old_dir); pwd()

"C:\\Bahiyaa\\Julia-Assignment\\Chapter 09"

In [29]:
fs = stat("Chapter-9 Standard Libraries.ipynb")

StatStruct(mode=0o100666, size=67866)

## 9.5 Parallel Programming

Execution of programs in tandem in various execution environments, yet be able to obtain meaningful and synchronized results. Performance and resource utilization are two major outcomes of this kind of programming. 

### Asynchronous Programming

Breaking the code into small chunks of executable pieces that can be executed in parallel. Depending on the kind of processing permitted by the hardware and OS, these may run in one execution thread or multiple.

#### Tasks

In [30]:
g = Task() do 
    sleep(20)
    println("done")
end

Task (runnable) @0x000000001050ea40

In [31]:
schedule(g)

Task (runnable) @0x000000001050ea40

In [32]:
g = Task() do 
    sleep(20)
    println("done")
end

Task (runnable) @0x000000000f7cad60

In [33]:
schedule(g); wait(g)

done
done


In [34]:
g = @task begin
    sleep(7)
    println("done")
end

Task (runnable) @0x000000000f7cb6c0

In [35]:
@sync  begin
    sleep(7)
    println("done")
end

done


#### Channels

In [36]:
a = Channel(8)

Channel{Any}(8) (empty)

In [37]:
a = Channel{Int}(7)

Channel{Int64}(7) (empty)

In [38]:
@async begin
    for j = 1:100
        println("Adding $j to channel")
        put!(a, j)
    end
end

@async begin
    for j = 1:100
        v = take!(a)
        println("Removing value: $v")
    end
end

Task (runnable) @0x000000000e80abd0

Adding 1 to channel
Adding 2 to channel

In [39]:
close(a)


Adding 3 to channel
Removing value: 1


### Multithreading

Julia can be made to run on a multiple threads mode. In such a conditions the tasks can be parallelized on these threads. Synchronization objects can be used across the executing threads. 

In [40]:
Threads.nthreads()

Removing value: 2


1

In [41]:
Threads.@spawn for j=1:100
    sleep(1)
    println("Step: $j")
end

Task (runnable) @0x000000000f9dd780

### Distributed Computing

This is cluster computing infrastrcuture, where a cluster can be configured across Julia processes in the same machine or in another machine. These independent processes can execute independently yet be able to share data, pass messages and communicate over a shared channel. 

## 9.6 IO and Network

Julia accesses input and output devices as streams. The network devices are also interfaced like sockets. IO is an important stream based interface to understand. 

### Default Stream

`stdin`, `stdout` and `stderr` are the standard streams for input, output and error in most processes. `stdin` integration with Jupyter is limited. All those samples may best be reviewed in a REPL console. 

In [42]:
write(stdout, "Welcome")

Welcome

7

In [43]:
write(stdout, "\u2200 a \u2203 b")

∀ a ∃ b

11

### Text I/O

### Binary I/O

### File I/O

Files are also accessed as `IO` streams for both read and write operations. However, the `IO` streams are obtained by `open` call. Any open stream must be closed with a `close` function call. 

In [121]:
write("hello.txt", "Welcome")

7

In [122]:
fd = open("hello.txt", "r");
data = read(fd);

In [124]:
close(fd)
write(stdout, data)

Welcome

7

In [125]:
open("hello.txt", "r") do fd
    data = read(fd)
    write(stdout, data)
end

Welcome

7

In [126]:
write("hello.txt", "Welcome");
data = read("hello.txt");
write(stdout, data);

Welcome

Files are opened for `read` or `write`. A file opened for reading cannot be written into.  

In [127]:
open("hello.txt", "r") do fd
    while !eof(fd)
        b = read(fd, 1)
        write(stdout, b)
    end
end
rm("hello.txt")

Welcome

### Delimited Files

Generalized command separated value (CSV) kind of files. Used extensively in spreadsheets. Also interfaced in data science applications as an easier way to share data. 

In [128]:
using DelimitedFiles
a = collect(1:100);
b = Float64.(collect(101:200));
open("values.csv", "w") do f
    println(f, "A,B")
    writedlm(f, [a b], ',')
end

In [130]:
data, header = readdlm("values.csv", ',', header=true);

In [131]:
size(data, 1)

100

In [132]:
header

1×2 Matrix{AbstractString}:
 "A"  "B"

In [133]:
rm("values.csv")

### Network 

Just like files are opened with open to obtain the `IO` stream to interface, sockets are equivalent network communication end points to obtain connection streams. However, if you could use download method to provide a URL and download files. 

In [56]:
download("https://github.com/JuliaLang/julia/releases/download/v1.5.3/julia-1.5.3.tar.gz", "julia-1.5.3.tar.gz")

Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
Step: 36
Step: 37
Step: 38
Step: 39
Step: 40
Step: 41
Step: 42
Step: 43
Step: 44
Step: 45
Step: 46
Step: 47
Step: 48
Step: 49
Step: 50
Step: 51
Step: 52
Step: 53
Step: 54
Step: 55
Step: 56
Step: 57
Step: 58
Step: 59
Step: 60
Step: 61
Step: 62
Step: 63
Step: 64
Step: 65
Step: 66
Step: 67
Step: 68
Step: 69
Step: 70
Step: 71
Step: 72
Step: 73
Step: 74
Step: 75
Step: 76
Step: 77
Step: 78
Step: 79
Step: 80
Step: 81
Step: 82
Step: 83
Step: 84
Step: 85
Step: 86
Step: 87
Step: 88
Step: 89
Step: 90
Step: 91
Step: 92
Step: 93
Step: 94
Step: 95
Step: 96
Step: 97
Step: 98
Step: 99
Step: 100


"julia-1.5.3.tar.gz"

In [134]:
using Sockets
@async begin
    server = listen(5000)
    while true
        e = accept(server)
        @async while isopen(e)
            msg = readline(e, keep=true)
            write(e, "Server Response: $msg")
        end  
    end
end

Task (runnable) @0x000000000e452bd0

In [135]:
b = connect(5000)
@async while isopen(b)
    println(stdout, readline(b, keep=true))
end

Task (runnable) @0x000000000f08c970

In [136]:
for j = 1:5
    println(b, "Client Msg id: $j")
end

Server Response: Client Msg id: 1

Server Response: Client Msg id: 2

Server Response: Client Msg id: 3

Server Response: Client Msg id: 4

Server Response: Client Msg id: 5



In [137]:
close(b)

### Memory I/O

Strings cannot be manipulated. It may help to create a byte array for `IO` in the memory and use that for text manipulations. In the end the byte array can be converted to a String. 

In [138]:
jo = IOBuffer()
println(jo, "This is Line 1")
println(jo, "This is Line 2")
string = String(take!(jo))

"This is Line 1\nThis is Line 2\n"

In [139]:
B = fill(10, (5, 20))
B[2, 3] = 20
open("mmap.bin", "w+") do fd
    write(fd, size(B, 1))
    write(fd, size(B, 2))
    write(fd, B)
end

800

Memory mapping is reverse problem. A file can be mapped to memory and accessed selectively page by page. This way the whole file need not be loaded in the memory. 

In [141]:
using Mmap
fd = open("mmap.bin", "r")
nr = read(fd, Int)
nc = read(fd, Int)
B2 = Mmap.mmap(fd, Matrix{Int}, (nr, nc))
println("B2[1, 1]: ", B2[1, 1], " B2[2, 3]: ", B2[2, 3])
close(fd)

B2[1, 1]: 10 B2[2, 3]: 20


## 9.7 Constants

There are many system and Julia related constants. Mathematical constants are also defined in the standard library. 

In [148]:
VERSION

v"1.6.7"

In [150]:
C_NULL

Ptr{Nothing} @0x0000000000000000

In [151]:
Sys.BINDIR

"C:\\Users\\zamee\\AppData\\Local\\Programs\\Julia-1.6.7\\bin"

In [152]:
Sys.CPU_THREADS

8

In [153]:
Sys.WORD_SIZE

64

In [154]:
Sys.KERNEL

:NT

In [155]:
Sys.MACHINE

"x86_64-w64-mingw32"

In [156]:
Sys.ARCH

:x86_64

In [157]:
Sys.ENV

Base.EnvDict with 55 entries:
  "=C:"                             => "C:\\Bahiyaa\\Julia-Assignment\\Chapter …
  "ALLUSERSPROFILE"                 => "C:\\ProgramData"
  "APPDATA"                         => "C:\\Users\\zamee\\AppData\\Roaming"
  "COLUMNS"                         => "80"
  "COMMONPROGRAMFILES"              => "C:\\Program Files\\Common Files"
  "COMMONPROGRAMFILES(X86)"         => "C:\\Program Files (x86)\\Common Files"
  "COMMONPROGRAMW6432"              => "C:\\Program Files\\Common Files"
  "COMPUTERNAME"                    => "LAPTOP-PLJR15VI"
  "COMSPEC"                         => "C:\\WINDOWS\\system32\\cmd.exe"
  "DRIVERDATA"                      => "C:\\Windows\\System32\\Drivers\\DriverD…
  "FPS_BROWSER_APP_PROFILE_STRING"  => "Internet Explorer"
  "FPS_BROWSER_USER_PROFILE_STRING" => "Default"
  "GOOGLE_API_KEY"                  => "no"
  "GOOGLE_DEFAULT_CLIENT_ID"        => "no"
  "GOOGLE_DEFAULT_CLIENT_SECRET"    => "no"
  "HOMEDRIVE"                       =

In [158]:
MathConstants.pi

π = 3.1415926535897...

In [159]:
MathConstants.e

ℯ = 2.7182818284590...

## 9.8 Notable Modules

Miscellaneous modules of Julia library. 

### Dates

User renderable date and time representations. In the system the date is just a continuously increasing counter from a datum. These functions make meaningful period associations and provide the correct date and time. 

In [160]:
using Dates

In [161]:
now()

2022-09-29T19:44:23.559

In [162]:
DateTime(2020)

2020-01-01T00:00:00

In [163]:
T = DateTime(2020, 10, 31, 01, 02, 03)

2020-10-31T01:02:03

In [164]:
Date(2020)

2020-01-01

In [165]:
d = Date(T)

2020-10-31

In [166]:
DateTime(d)

2020-10-31T00:00:00

In [167]:
T > d

true

In [168]:
DateTime(2020) < now()

true

In [169]:
DateTime(2020) == Date(2020)

true

In [170]:
q = Month(3) + Day(10) + Hour(15)

3 months, 10 days, 15 hours

In [171]:
typeof(q)

Dates.CompoundPeriod

In [172]:
now() + q

2023-01-09T10:45:11.429

In [173]:
Date(now())+ Day(300)

2023-07-26

In [174]:
now() - DateTime(2020)

86643917138 milliseconds

In [177]:
Date(now()) - Date(2020)

1002 days

In [178]:
Year(now())

2022 years

In [179]:
Month(now())

9 months

In [180]:
Day(now())

29 days

In [181]:
Week(now())

39 weeks

In [182]:
year(now())

2022

In [183]:
tonext(today()) do b
    dayofweek(b) == Thursday &&
    dayofweekofmonth(b) == 4 &&
    month(b) == November
end

2022-11-24

### Logging

Long running processes or servers running in Julia need to provide consistent and standard log reporting. This module helps create such reports. 

In [184]:
@debug "This is a debug message $(sum(rand(100)))"

In [185]:
@info "This message is just informational"

┌ Info: This message is just informational
└ @ Main In[185]:1


In [186]:
@warn "This is a warning message"

┌ Warning: This is a warning message
└ @ Main In[186]:1


In [187]:
@error "This is an error message"

┌ Error: This is an error message
└ @ Main In[187]:1


In [188]:
C = ones(5, 6)
@info "C is all ones" C

┌ Info: C is all ones
│   C = [1.0 1.0 1.0 1.0 1.0 1.0; 1.0 1.0 1.0 1.0 1.0 1.0; 1.0 1.0 1.0 1.0 1.0 1.0; 1.0 1.0 1.0 1.0 1.0 1.0; 1.0 1.0 1.0 1.0 1.0 1.0]
└ @ Main In[188]:2


The `Logging` module provides interfaces to implement complete loggers. It also provides the ability to override the global logger with a different local one. 

In [189]:
using Logging

In [190]:
open("file.log", "w") do v
    with_logger(SimpleLogger(v)) do
        @info "This is my simple info log"
        @error "This is an error written to file"
        @warn "This is a warning message"
        @info "Current logger is same as global logger" current_logger() == global_logger()
    end
end
write(stdout, read("file.log"));

┌ Info: This is my simple info log
└ @ Main In[190]:3
┌ Error: This is an error written to file
└ @ Main In[190]:4
┌ Warning: This is a warning message
└ @ Main In[190]:5
┌ Info: Current logger is same as global logger
│   current_logger() == global_logger() = false
└ @ Main In[190]:6


In [191]:
rm("file.log")

In [192]:
@info "Current logger is same as global logger" current_logger() == global_logger()

┌ Info: Current logger is same as global logger
│   current_logger() == global_logger() = true
└ @ Main In[192]:1


### Statistics

This module provides simple functions for central tendencies like mean, median and standard deviations etc. Functions for correlations and covariances are also provided. However, the treatment of statistics in thi module are highly rudimentary. 

In [193]:
using Statistics

In [194]:
mean(1:20)

10.5

In [195]:
median(1:20)

10.5

In [196]:
std(1:20)

5.916079783099616

In [197]:
var(1:20)

35.0

In [198]:
cor(1:20, rand(20))

-0.27940201992764324

In [201]:
cor(1:20, 11:30)

1.0

In [202]:
cov(1:20, rand(20))

0.4046815050480789

### Random Numbers

Provides methods for random number generation and other permutation and combination functions. 

In [203]:
using Random

In [205]:
rand(Int, 3)

3-element Vector{Int64}:
  4358597447578198679
   701028325588804875
 -7849777488490615546

In [207]:
rand(4, 3)

4×3 Matrix{Float64}:
 0.0637732  0.73565    0.764657
 0.702085   0.0874664  0.0171868
 0.94724    0.729542   0.148757
 0.420911   0.982016   0.289751

In [208]:
rand!(zeros(4, 3))

4×3 Matrix{Float64}:
 0.12985    0.0103503  0.322221
 0.357087   0.504799   0.341217
 0.989176   0.87307    0.543285
 0.0646752  0.996353   0.772184

In [209]:
rand(1:4, (4, 3))

4×3 Matrix{Int64}:
 4  3  4
 3  3  3
 2  3  3
 1  1  4

## 9.9 Conclusion

## Exercises